# Ensemble 202108

#### 2.1  Seteo del ambiente en Google Colab

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/dmeyf"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/dmeyf" /content/buckets/b1


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/competencia_02_crudo.csv.gz"
archivo_destino="/content/datasets/competencia_02_crudo.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/competencia_02_crudo.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi


## Inicializacion

In [1]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-11-16 23:15:59 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,658286,35.2,1454468,77.7,1346869,72.0
Vcells,1222635,9.4,8388608,64.0,1975128,15.1


In [2]:
PARAM <- list()
PARAM$experimento <- "zensemble-202108"
PARAM$semilla_primigenia <- 202513

In [3]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### Ensemble

In [4]:
library(data.table)

In [12]:
PARAM$foto_mes_filtro <- 202108

In [13]:
modelos_info <- data.table(

  experimento_id = c("seg-001",
                     "zmuerte-10-2",
                     "zmuerte-50-2",
                     "apo-006"),

  score = c(399999,
            385599,
            388799,
            399759),

  archivo_pred = c("prediccion.txt",
                   "prediccion.txt",
                   "prediccion.txt",
                   "prediccion.txt")
)

RUTA_BASE_EXP <- "/content/buckets/b1/exp/"

# Weights

score_total <- sum(modelos_info$score)
modelos_info[, peso := score / score_total]

# Ensamble

tb_ensamble <- NULL


for (i in 1:nrow(modelos_info)) {

  modelo_actual <- modelos_info[i]

  # Construyo la ruta completa al archivo de predicción
  ruta_completa <- paste0(
    RUTA_BASE_EXP,
    modelo_actual$experimento_id, "/",
    modelo_actual$archivo_pred
  )

  tryCatch({
    # Leo el archivo de predicción
    tb_pred <- fread(ruta_completa)

    # Verifico que tenga las columnas esperadas
    if (!all(c("numero_de_cliente", "prob") %in% names(tb_pred))) {
      next
    }

    # Filtro Mes
    tb_pred <- tb_pred[foto_mes == PARAM$foto_mes_filtro]

    # Si después del filtro aún hay duplicados, nos quedamos con la primera aparición
    tb_pred <- unique(tb_pred, by = "numero_de_cliente")

    # Renombro la columna 'prob' para que sea única de este modelo
    col_prob_nueva <- paste0("prob_", modelo_actual$experimento_id)
    setnames(tb_pred, "prob", col_prob_nueva)

    # Me quedo solo con las columnas que importan (ID y prob)
    tb_pred <- tb_pred[, c("numero_de_cliente", col_prob_nueva), with = FALSE]

    # --- Merge ---
    if (is.null(tb_ensamble)) {
      # Es el primer archivo, lo uso como base
      tb_ensamble <- tb_pred
    } else {
      # Hago merge con los IDs para alinear las predicciones
      tb_ensamble <- merge(tb_ensamble, tb_pred, by = "numero_de_cliente", all = TRUE)
    }

  }, error = function(e) {
    cat("  ERROR al leer el archivo:", ruta_completa, "-", e$message, "\n")
  })
}

# Ensamble Ponderado

if (!is.null(tb_ensamble)) {

  tb_ensamble[, prob_ensamble := 0.0]

  for (i in 1:nrow(modelos_info)) {

    modelo_actual <- modelos_info[i]
    col_prob <- paste0("prob_", modelo_actual$experimento_id)
    peso_actual <- modelo_actual$peso

    if (col_prob %in% names(tb_ensamble)) {

      tb_ensamble[, prob_ensamble := prob_ensamble + (get(col_prob) * peso_actual), ]
    }
  }
}

  # Me quedo solo con la predicción final
tb_final <- tb_ensamble[, .(
  numero_de_cliente,
  foto_mes    = PARAM$foto_mes_filtro,
  meta_modelo = 1L,
  prob        = prob_ensamble,
  gan_acum    = NA_real_
)]

  # Aseguramos que solo haya una fila por cliente
tb_final <- tb_final[
  order(-prob),
  .SD[1],
  by = numero_de_cliente
]


  fwrite(tb_final,
    file= "prediccion.txt", # <=== Este es el nuevo archivo TXT
    sep= "\t"
  )

### Clasificacion

In [14]:
# genero archivos con los  "envios" mejores
dir.create("kaggle", showWarnings=FALSE)

# Tomo solo las filas del período elegido
tb_kaggle <- tb_final[foto_mes == PARAM$foto_mes_filtro]

# ordeno por probabilidad descendente
setorder(tb_kaggle, -prob)

envios <- 11000
tb_kaggle[, Predicted := 0L] # seteo inicial a 0
tb_kaggle[1:envios, Predicted := 1L] # marco los primeros

archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

# grabo el archivo
fwrite(tb_kaggle[, list(numero_de_cliente, Predicted)],
  file= archivo_kaggle,
  sep= ","
)

In [15]:
Sys.time()

[1] "2025-11-16 23:30:51 UTC"